**Importing Required Packages**

In [1]:
import torch
import torchvision
from torch import nn, optim

import torch.nn.functional as F
from torchsummary import summary

**Configuring Model**

In [2]:
# Model Configs
batch_size = 64
learning_rate = 0.01
cross_entropy = nn.CrossEntropyLoss()

**Load MNIST Data**

In [3]:
# Data Loader
transform = torchvision.transforms.ToTensor()
train_data = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
    'mnist_data', train=True, download=True, transform=transform
    ), batch_size=batch_size
)
val_data = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
    'mnist_data', train=False, download=True, transform=transform
    ), batch_size=batch_size
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


**Define Validation Function**

Function to calculate accuracy of the given validation data

In [4]:
# Validation function
def validate(model, data):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(data):
        images = images.cuda()
        labels = labels.cuda()
        y_pred = model(images)
        value, pred = torch.max(y_pred, 1)
        total += y_pred.size(0)
        correct += torch.sum(pred == labels)
    return correct * 100 / total

**Define Training Function**

Function to train the model on the training data

In [5]:
# Training Function
def train(model,epochs=5) :
    optimizer = optim.Adam(model.parameters(),lr=learning_rate)    
    for n in range(epochs)  :
        for i , (images , labels) in enumerate(train_data) :
            images = images.cuda()
            labels = labels.cuda()
            optimizer.zero_grad()
            prediction = model(images)
            loss = cross_entropy(prediction, labels)
            loss.backward()
            optimizer.step()
        accuracy = float(validate(model, val_data))
        print("Epoch:", n+1, "Loss: ", float(loss.data), "Accuracy:", accuracy)

**Define Model**

In [9]:
# Model
class CNNWithPoolRelu(nn.Module) :
    def __init__(self):
        super(CNNWithPoolRelu,self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3)
        self.conv_2 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3)
        
        self.pool_1 = nn.MaxPool2d(2)
        self.pool_2 = nn.MaxPool2d(2)

        self.dense_1 = nn.Linear(in_features=800,out_features=256)
        self.dense_2 = nn.Linear(in_features=256,out_features=10)

        self.relu = nn.ReLU()
    def forward(self,x) :
        x = self.relu(self.conv_1(x))
        x = self.pool_1(x)
        x = self.relu(self.conv_2(x))
        x = self.pool_2(x)
        x = x.view(x.shape[0],-1)
        x = self.relu(self.dense_1(x))
        x = self.dense_2(x)
        # output = self.tanh(x)
        output = F.log_softmax(x, dim=1)

        return output

**Create Model Instance**

In [10]:
# Model
model = CNNWithPoolRelu().cuda()

**Model Summary**

In [11]:
# Summary
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
         MaxPool2d-3           [-1, 16, 13, 13]               0
            Conv2d-4           [-1, 32, 11, 11]           4,640
              ReLU-5           [-1, 32, 11, 11]               0
         MaxPool2d-6             [-1, 32, 5, 5]               0
            Linear-7                  [-1, 256]         205,056
              ReLU-8                  [-1, 256]               0
            Linear-9                   [-1, 10]           2,570
Total params: 212,426
Trainable params: 212,426
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.25
Params size (MB): 0.81
Estimated Total Size (MB): 1.07
-------------------------------------------

**Train the model**

In [12]:
# Train for 30 Epochs
train(model,epochs=30)

Epoch: 1 Loss:  0.15816104412078857 Accuracy: 94.64999389648438
Epoch: 2 Loss:  0.1524432897567749 Accuracy: 93.48999786376953
Epoch: 3 Loss:  0.09196223318576813 Accuracy: 97.02999877929688
Epoch: 4 Loss:  0.04454255476593971 Accuracy: 96.37999725341797
Epoch: 5 Loss:  0.0044022477231919765 Accuracy: 97.27999877929688
Epoch: 6 Loss:  0.0051766070537269115 Accuracy: 97.81999969482422
Epoch: 7 Loss:  0.1392780989408493 Accuracy: 97.94999694824219
Epoch: 8 Loss:  0.015993306413292885 Accuracy: 97.22999572753906
Epoch: 9 Loss:  0.01502278447151184 Accuracy: 97.70999908447266
Epoch: 10 Loss:  0.011916162446141243 Accuracy: 97.5
Epoch: 11 Loss:  0.0043948860839009285 Accuracy: 97.70999908447266
Epoch: 12 Loss:  0.0010469170520082116 Accuracy: 97.69999694824219
Epoch: 13 Loss:  0.0021841730922460556 Accuracy: 97.63999938964844
Epoch: 14 Loss:  0.0014315568841993809 Accuracy: 97.05999755859375
Epoch: 15 Loss:  0.0022074480075389147 Accuracy: 97.8699951171875
Epoch: 16 Loss:  0.001716332277283

We can see that using ReLU function gives us better accuracy and is able to reduce loss more effectively than tanh.